## Import Libraries 

In [1]:
# Import libraries
from datetime import timedelta
import os

import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler


from IPython.display import display, HTML, Image
%matplotlib inline

plt.style.use('ggplot')
plt.rcParams.update({'font.size': 20})

# Access data using Google BigQuery.
# from google.colab import auth
from google.cloud import bigquery
import bigframes.pandas as bf

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, GRU

import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

/Users/sadhikrupa/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
vital_signs_df = pd.read_csv("signs_with_AGE_all.csv")
lab_value_wc = pd.read_csv("values_all.csv")
lab_value_ph = pd.read_csv("PH_all.csv")

In [3]:
vital_signs_df['subject_id'].value_counts()

subject_id
16454913    4615
14411859    3690
18295542    3566
15689523    3301
10858147    3204
            ... 
17320563       6
15286220       6
19155097       5
11234187       5
12758384       4
Name: count, Length: 4997, dtype: int64

In [4]:
# Merge on subject_id, hadm_id, stay_id, and charttime
combined_df = pd.merge(lab_value_wc,lab_value_ph, on=['subject_id', 'hadm_id',  'charttime'], how='outer')

# Sort the combined DataFrame by subject_id and charttime
combined_df = combined_df.sort_values(by=['subject_id', 'charttime'])

In [5]:
# Merge the DataFrames on subject_id, hadm_id, and charttime
df = pd.merge(vital_signs_df, combined_df, on=['subject_id', 'hadm_id', 'charttime'], how='outer')

# Sort by subject_id and charttime
df = df.sort_values(by=['subject_id', 'charttime'])

### Data frame

In [6]:
df

,subject_id,hadm_id,stay_id,charttime,age,heart_rate,temperature,resp_rate,systolic_bp,diastolic_bp,blood_oxygen_saturation,co2_partial_pressure,white_blood_cells,ph_value
328,10002428,28662225,33987268.0,2156-04-12 17:25:00,80.0,NaN,102.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
329,10002428,28662225,33987268.0,2156-04-12 17:26:00,80.0,124.0,NaN,25.0,NaN,NaN,90.0,NaN,NaN,NaN
330,10002428,28662225,33987268.0,2156-04-12 17:27:00,80.0,NaN,NaN,NaN,87.0,42.0,NaN,NaN,NaN,NaN
331,10002428,28662225,33987268.0,2156-04-12 17:30:00,80.0,120.0,NaN,29.0,94.0,37.0,93.0,NaN,NaN,NaN
332,10002428,28662225,33987268.0,2156-04-12 17:45:00,80.0,134.0,102.9,22.0,92.0,51.0,94.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1370581,19997367,21508795,NaN,2127-04-13 05:52:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN
1370582,19997367,21508795,NaN,2127-04-14 06:42:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.4,NaN
1370583,19997367,21508795,NaN,2127-04-15 06:50:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN
1370584,19997367,21508795,NaN,2127-04-16 10:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.5,NaN


In [7]:
# Aggregate data into hourly intervals
df['charttime'] = pd.to_datetime(df['charttime'])

df.set_index('charttime', inplace=True)

## Time series Data Analysis 

#### Hourly interval data 

In [8]:
# Step : 1 Resample the data to hourly intervals, calculating the mean for each hour
hourly_df = df.groupby(['subject_id', 'hadm_id', 'stay_id']).resample('1H').mean()

/var/folders/xg/f264hrzn7j7gdpk7gdvq7ffh0000gn/T/ipykernel_84176/3592301732.py:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hourly_df = df.groupby(['subject_id', 'hadm_id', 'stay_id']).resample('1H').mean()


In [9]:
hourly_df.head()

subject_id     hadm_id  \
subject_id hadm_id  stay_id    charttime                                     
10002428   23473524 35479615.0 2156-05-11 16:00:00  10002428.0  23473524.0   
                               2156-05-11 17:00:00  10002428.0  23473524.0   
                               2156-05-11 18:00:00  10002428.0  23473524.0   
                               2156-05-11 19:00:00  10002428.0  23473524.0   
                               2156-05-11 20:00:00  10002428.0  23473524.0   

                                                       stay_id   age  \
subject_id hadm_id  stay_id    charttime                               
10002428   23473524 35479615.0 2156-05-11 16:00:00  35479615.0  80.0   
                               2156-05-11 17:00:00  35479615.0  80.0   
                               2156-05-11 18:00:00  35479615.0  80.0   
                               2156-05-11 19:00:00  35479615.0  80.0   
                               2156-05-11 20:00:00  35479615.0  80.0   

                                                    heart_rate  temperature  \
subject_id hadm_id  stay_id    charttime                                      
10002428   23473524 35479615.0 2156-05-11 16:00:00        68.2          NaN   
                               2156-05-11 17:00:00        68.0         97.2   
                               2156-05-11 18:00:00        73.0          NaN   
                               2156-05-11 19:00:00        85.0          NaN   
                               2156-05-11 20:00:00       102.0         97.7   

                                                    resp_rate  systolic_bp  \
subject_id hadm_id  stay_id    charttime                                     
10002428   23473524 35479615.0 2156-05-11 16:00:00       17.8        128.2   
                               2156-05-11 17:00:00       18.0        106.0   
                               2156-05-11 18:00:00       18.0        122.0   
                               2156-05-11 19:00:00       18.0        103.0   
                               2156-05-11 20:00:00       19.0         99.0   

                                                    diastolic_bp  \
subject_id hadm_id  stay_id    charttime                           
10002428   23473524 35479615.0 2156-05-11 16:00:00          65.2   
                               2156-05-11 17:00:00          58.0   
                               2156-05-11 18:00:00          63.0   
                               2156-05-11 19:00:00          51.0   
                               2156-05-11 20:00:00          52.0   

                                                    blood_oxygen_saturation  \
subject_id hadm_id  stay_id    charttime                                      
10002428   23473524 35479615.0 2156-05-11 16:00:00                    100.0   
                               2156-05-11 17:00:00                    100.0   
                               2156-05-11 18:00:00                    100.0   
                               2156-05-11 19:00:00                     99.0   
                               2156-05-11 20:00:00                     98.0   

                                                    co2_partial_pressure  \
subject_id hadm_id  stay_id    charttime                                   
10002428   23473524 35479615.0 2156-05-11 16:00:00                   NaN   
                               2156-05-11 17:00:00                  46.0   
                               2156-05-11 18:00:00                   NaN   
                               2156-05-11 19:00:00                   NaN   
                               2156-05-11 20:00:00                   NaN   

                                                    white_blood_cells  \
subject_id hadm_id  stay_id    charttime                                
10002428   23473524 35479615.0 2156-05-11 16:00:00                NaN   
                               2156-05-11 17:00:00                NaN   
                               2156-05-11 1

In [10]:
# Check if the columns already exist, and if so, drop them before resetting the index
hourly_df = hourly_df.drop(columns=['stay_id', 'subject_id', 'hadm_id'], errors='ignore').reset_index()


In [11]:
hourly_df = hourly_df.reset_index()

###  Handling missing Value using carry forward and backward imputation 

In [12]:

# Step 2: Implement carry forward/backward imputation
hourly_df = hourly_df.groupby(['subject_id', 'hadm_id', 'stay_id']).apply(lambda group: group.fillna(method='ffill').fillna(method='bfill'))


/var/folders/xg/f264hrzn7j7gdpk7gdvq7ffh0000gn/T/ipykernel_84176/1621308365.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  hourly_df = hourly_df.groupby(['subject_id', 'hadm_id', 'stay_id']).apply(lambda group: group.fillna(method='ffill').fillna(method='bfill'))
/var/folders/xg/f264hrzn7j7gdpk7gdvq7ffh0000gn/T/ipykernel_84176/1621308365.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  hourly_df = hourly_df.groupby(['subject_id', 'hadm_id', 'stay_id']).apply(lambda group: group.fillna(method='ffill').fillna(method='bfill'))


In [13]:
hourly_df = hourly_df.drop(columns=['stay_id', 'subject_id', 'hadm_id'], errors='ignore').reset_index()

### Sepsis Defination 1 

In [14]:
def check_sirs_criteria(row):
    criteria_met = 0
    
    # Temperature abnormality in Fahrenheit
    if row['temperature'] < 96.8 or row['temperature'] > 100.4:
        criteria_met += 1
    
    # Heart rate abnormality
    if row['heart_rate'] > 90:
        criteria_met += 1
    
    # Respiratory rate abnormality
    if row['resp_rate'] > 20:
        criteria_met += 1
    
    # White blood cell count abnormality (using 1000s for the range of normal values)
    if pd.notna(row['white_blood_cells']):  # Check if WBC value is not null
        if row['white_blood_cells'] < 4.0 or row['white_blood_cells'] > 12.0:
            criteria_met += 1
    
    # Return whether 2 or more criteria are met
    return 1 if criteria_met >= 2 else 0

### Checking 5 hr of Window if two above feature, high for patients, their first hour called speis_onset

In [15]:

# Step 3: Apply the 5-hour SIRS window check

# hourly_df = hourly_df.groupby(['subject_id', 'hadm_id', 'stay_id']).apply(check_sirs_window)
def check_sirs_window(df):
    df['sirs_criteria_met'] = df.apply(check_sirs_criteria, axis=1)
    df['sirs_window'] = df['sirs_criteria_met'].rolling(window=5).sum()
    
    # Mark the sepsis onset at the first hour of the 5-hour interval
    df['sepsis_onset'] = 0
    df.loc[df['sirs_window'] >= 5, 'sepsis_onset'] = 1
    
    # Shift sepsis onset to the first hour of the 5-hour window
    df['sepsis_onset'] = df['sepsis_onset'].shift(-4).fillna(0).astype(int)
    
    return df

hourly_df = hourly_df.groupby(['subject_id', 'hadm_id', 'stay_id']).apply(check_sirs_window)


/var/folders/xg/f264hrzn7j7gdpk7gdvq7ffh0000gn/T/ipykernel_84176/3420506990.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  hourly_df = hourly_df.groupby(['subject_id', 'hadm_id', 'stay_id']).apply(check_sirs_window)


In [16]:
hourly_df 

subject_id   hadm_id     stay_id  \
subject_id hadm_id  stay_id                                               
10002428   23473524 35479615.0 0         10002428  23473524  35479615.0   
                               1         10002428  23473524  35479615.0   
                               2         10002428  23473524  35479615.0   
                               3         10002428  23473524  35479615.0   
                               4         10002428  23473524  35479615.0   
...                                           ...       ...         ...   
19997367   21508795 36980198.0 906461    19997367  21508795  36980198.0   
                               906462    19997367  21508795  36980198.0   
                               906463    19997367  21508795  36980198.0   
                               906464    19997367  21508795  36980198.0   
                               906465    19997367  21508795  36980198.0   

                                       level_3   index           charttime  \
subject_id hadm_id  stay_id                                                  
10002428   23473524 35479615.0 0             0       0 2156-05-11 16:00:00   
                               1             1       1 2156-05-11 17:00:00   
                               2             2       2 2156-05-11 18:00:00   
                               3             3       3 2156-05-11 19:00:00   
                               4             4       4 2156-05-11 20:00:00   
...                                        ...     ...                 ...   
19997367   21508795 36980198.0 906461   906461  906461 2127-04-06 16:00:00   
                               906462   906462  906462 2127-04-06 17:00:00   
                               906463   906463  906463 2127-04-06 18:00:00   
                               906464   906464  906464 2127-04-06 19:00:00   
                               906465   906465  906465 2127-04-06 20:00:00   

                                        age  heart_rate  temperature  \
subject_id hadm_id  stay_id                                            
10002428   23473524 35479615.0 0       80.0        68.2         97.2   
                               1       80.0        68.0         97.2   
                               2       80.0        73.0         97.2   
                               3       80.0        85.0         97.2   
                               4       80.0       102.0         97.7   
...                                     ...         ...          ...   
19997367   21508795 36980198.0 906461  63.0        98.0         97.3   
                               906462  63.0        98.0         97.3   
                               906463  63.0        98.0         97.3   
                               906464  63.0       102.0         97.3   
                               906465  63.0       103.0         98.1   

                                       resp_rate  systolic_bp  diastolic_bp  \
subject_id hadm_id  stay_id                                                   
10002428   23473524 35479615.0 0            17.8        128.2          65.2   
                               1            18.0        106.0          58.0   
                               2            18.0        122.0          63.0   
                               3            18.0        103.0          51.0   
                               4            19.0         99.0          52.0   
...                                          ...          ...           ...   
19997367   21508795 36980198.0 906461       18.0         99.0          55.0   
                               906462       17.0         99.0          55.0   
                               906463       17.0        101.0          48.0   
                               906464       21.0         99.0          48.0   
                               906465       20.0         99.0          87.0   

                                       blood_oxygen_saturation  \
subject_id hadm_i

In [17]:
# Check if the columns already exist, and if so, drop them before resetting the index
hourly_df = hourly_df.drop(columns=['stay_id', 'subject_id', 'hadm_id','level_3'], errors='ignore').reset_index()

In [18]:
hourly_df.drop(columns = ['sirs_window','sirs_criteria_met'],inplace = True)

In [19]:
hourly_df

,subject_id,hadm_id,stay_id,level_3,index,charttime,age,heart_rate,temperature,resp_rate,systolic_bp,diastolic_bp,blood_oxygen_saturation,co2_partial_pressure,white_blood_cells,ph_value,sepsis_onset
0,10002428,23473524,35479615.0,0,0,2156-05-11 16:00:00,80.0,68.2,97.2,17.8,128.2,65.2,100.0,46.0,NaN,7.35,0
1,10002428,23473524,35479615.0,1,1,2156-05-11 17:00:00,80.0,68.0,97.2,18.0,106.0,58.0,100.0,46.0,NaN,7.35,0
2,10002428,23473524,35479615.0,2,2,2156-05-11 18:00:00,80.0,73.0,97.2,18.0,122.0,63.0,100.0,46.0,NaN,7.35,0
3,10002428,23473524,35479615.0,3,3,2156-05-11 19:00:00,80.0,85.0,97.2,18.0,103.0,51.0,99.0,46.0,NaN,7.35,0
4,10002428,23473524,35479615.0,4,4,2156-05-11 20:00:00,80.0,102.0,97.7,19.0,99.0,52.0,98.0,46.0,NaN,7.35,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
906461,19997367,21508795,36980198.0,906461,906461,2127-04-06 16:00:00,63.0,98.0,97.3,18.0,99.0,55.0,96.0,NaN,7.6,NaN,0
906462,19997367,21508795,36980198.0,906462,906462,2127-04-06 17:00:00,63.0,98.0,97.3,17.0,99.0,55.0,99.0,NaN,7.6,NaN,0
906463,19997367,21508795,36980198.0,906463,906463,2127-04-06 18:00:00,63.0,98.0,97.3,17.0,101.0,48.0,98.0,NaN,7.6,NaN,0
906464,19997367,21508795,36980198.0,906464,906464,2127-04-06 19:00:00,63.0,102.0,97.3,21.0,99.0,48.0,98.0,NaN,7.6,NaN,0


In [20]:
hourly_df = hourly_df.drop(columns=['level_3','index'], errors='ignore')

### Normalize the data 

In [21]:
# Step 4: Normalize the data
scaler = MinMaxScaler()
numerical_columns = ['age','heart_rate', 'temperature', 'resp_rate', 'systolic_bp', 'diastolic_bp', 'blood_oxygen_saturation', 'co2_partial_pressure', 'white_blood_cells', 'ph_value']
hourly_df[numerical_columns] = scaler.fit_transform(hourly_df[numerical_columns])


### Now, we are replacing Nan with 0 - ( lstm, Rnn we are padding nan with 0 ) 

We’re taking this approach because, in time series analysis, every piece of patient data is crucial. Padding missing values with zeros doesn’t negatively impact performance and allows the model to learn from other patients’ data as well. For example, if a patient’s temperature is missing but their white blood cell count is elevated, this is still valuable information. Removing data points solely due to missing temperature values could lead the model to overlook critical patterns.

In [22]:
hourly_df.fillna(0, inplace=True)

### Sepsis onset 

In [23]:
hourly_df['sepsis_onset'].value_counts()

sepsis_onset
0    623866
1    282600
Name: count, dtype: int64

### Filttering the subject Id we required at least 130

In [24]:

subject_counts = hourly_df['subject_id'].value_counts()

valid_subject_ids = subject_counts[subject_counts >= 130].index

filtered_hourly_df = hourly_df[hourly_df['subject_id'].isin(valid_subject_ids)]

print(f"Original dataframe shape: {hourly_df.shape}")
print(f"Filtered dataframe shape: {filtered_hourly_df.shape}")


Original dataframe shape: (906466, 15)
Filtered dataframe shape: (738306, 15)


In [25]:
filtered_hourly_df['subject_id'].value_counts()

subject_id
16454913    3559
18295542    2994
14411859    2689
15689523    2209
10670818    1958
            ... 
13670977     130
11305002     130
18085171     130
12115948     130
13114866     130
Name: count, Length: 1930, dtype: int64

In [26]:
hourly_df = filtered_hourly_df.copy()

### Final Dataframe

In [27]:
hourly_df

,subject_id,hadm_id,stay_id,charttime,age,heart_rate,temperature,resp_rate,systolic_bp,diastolic_bp,blood_oxygen_saturation,co2_partial_pressure,white_blood_cells,ph_value,sepsis_onset
0,10002428,23473524,35479615.0,2156-05-11 16:00:00,0.849315,0.007861,0.097395,0.000075,0.001104,0.000686,0.000200,0.017809,0.000000,0.5875,0
1,10002428,23473524,35479615.0,2156-05-11 17:00:00,0.849315,0.007851,0.097395,0.000076,0.000913,0.000610,0.000200,0.017809,0.000000,0.5875,0
2,10002428,23473524,35479615.0,2156-05-11 18:00:00,0.849315,0.008102,0.097395,0.000076,0.001051,0.000662,0.000200,0.017809,0.000000,0.5875,0
3,10002428,23473524,35479615.0,2156-05-11 19:00:00,0.849315,0.008706,0.097395,0.000076,0.000887,0.000536,0.000198,0.017809,0.000000,0.5875,0
4,10002428,23473524,35479615.0,2156-05-11 20:00:00,0.849315,0.009562,0.097896,0.000080,0.000853,0.000547,0.000196,0.017809,0.000000,0.5875,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
906461,19997367,21508795,36980198.0,2127-04-06 16:00:00,0.616438,0.009360,0.097495,0.000076,0.000853,0.000578,0.000192,0.000000,0.034771,0.0000,0
906462,19997367,21508795,36980198.0,2127-04-06 17:00:00,0.616438,0.009360,0.097495,0.000071,0.000853,0.000578,0.000198,0.000000,0.034771,0.0000,0
906463,19997367,21508795,36980198.0,2127-04-06 18:00:00,0.616438,0.009360,0.097495,0.000071,0.000870,0.000505,0.000196,0.000000,0.034771,0.0000,0
906464,19997367,21508795,36980198.0,2127-04-06 19:00:00,0.616438,0.009562,0.097495,0.000088,0.000853,0.000505,0.000196,0.000000,0.034771,0.0000,0


## Time Series Analysis

sequences, labels

In [28]:
def create_sequences_with_variable_prediction_time(data, window_size, prediction_time):
    sequences = []
    labels = []
    
    # Group by subject_id to ensure sequences are generated separately for each subject
    for subject_id, group in data.groupby('subject_id'):
        group = group.reset_index(drop=True)  # Reset index for each subject
        
        # Drop non-relevant columns
        group = group.drop(columns=['subject_id', 'hadm_id', 'stay_id', 'charttime', 'sepsis_onset'])
        
        for i in range(len(group) - window_size - prediction_time):
            sequence = group.iloc[i:i+window_size].values  # Get look-back window
            label = data['sepsis_onset'].iloc[i+window_size + prediction_time - 1]  # Predict n hours ahead
            sequences.append(sequence)
            labels.append(label)
    
    return np.array(sequences), np.array(labels)

###  Based on 5 hr data , predict 3 hr before speis onset in patients

In [29]:
window_size = 5  # Look-back period
prediction_time = 3  # Predict 3 hours ahead

sequences, labels = create_sequences_with_variable_prediction_time(hourly_df, window_size, prediction_time)



In [30]:
sequences

array([[[0.84931507, 0.0078607 , 0.09739479, ..., 0.01780875,
         0.        , 0.5875    ],
        [0.84931507, 0.00785064, 0.09739479, ..., 0.01780875,
         0.        , 0.5875    ],
        [0.84931507, 0.00810226, 0.09739479, ..., 0.01780875,
         0.        , 0.5875    ],
        [0.84931507, 0.00870615, 0.09739479, ..., 0.01780875,
         0.        , 0.5875    ],
        [0.84931507, 0.00956167, 0.09789579, ..., 0.01780875,
         0.        , 0.5875    ]],

       [[0.84931507, 0.00785064, 0.09739479, ..., 0.01780875,
         0.        , 0.5875    ],
        [0.84931507, 0.00810226, 0.09739479, ..., 0.01780875,
         0.        , 0.5875    ],
        [0.84931507, 0.00870615, 0.09739479, ..., 0.01780875,
         0.        , 0.5875    ],
        [0.84931507, 0.00956167, 0.09789579, ..., 0.01780875,
         0.        , 0.5875    ],
        [0.84931507, 0.00956167, 0.09789579, ..., 0.01780875,
         0.        , 0.5875    ]],

       [[0.84931507, 0.00810226, 0.0

In [31]:
labels

array([0, 0, 0, ..., 0, 0, 0])

## GRU Model 

In [32]:
# Define the GRU model
model = Sequential()

# Add an GRU layer
model.add(GRU(units=64, return_sequences=False, input_shape=(window_size, sequences.shape[2])))

# Add a Dense output layer for binary classification
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

2024-10-28 23:53:34.444308: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Max
2024-10-28 23:53:34.444330: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 36.00 GB
2024-10-28 23:53:34.444334: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 13.50 GB
2024-10-28 23:53:34.444350: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-10-28 23:53:34.444362: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/Users/sadhikrupa/Library/Python/3.9/lib/python/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using 

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru (GRU)                       │ (None, 64)             │        14,592 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,657 (57.25 KB)

 Trainable params: 14,657 (57.25 KB)

 Non-trainable params: 0 (0.00 B)

In [33]:
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
import numpy as np

def create_cross_validation_splits(sequences, labels, n_splits=4):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    for train_index, test_index in skf.split(sequences, labels):
        X_train, X_test = sequences[train_index], sequences[test_index]
        y_train, y_test = labels[train_index], labels[test_index]

        # Split training data into training and validation (9/12 training, 3/12 validation)
        X_train_full, X_val, y_train_full, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)
        
        # Normalize the data (ensure consistent normalization across folds)
        scaler = MinMaxScaler()
        X_train_full = scaler.fit_transform(X_train_full.reshape(-1, X_train_full.shape[-1])).reshape(X_train_full.shape)
        X_val = scaler.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)
        X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)
        
        yield X_train_full, X_val, X_test, y_train_full, y_val, y_test

## Class Weight Balancing 

In [34]:

# Split the sequences and labels into training, validation, and testing sets
X_train_full, X_test, y_train_full, y_test = train_test_split(sequences, labels, test_size=0.2, random_state=42)

# Split the training data again into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42)

# Calculate class weights based on the training data
class_weights = class_weight.compute_class_weight('balanced',
                                                  classes=np.unique(y_train),
                                                  y=y_train)

class_weight_dict = dict(enumerate(class_weights))



In [35]:
# Define early stopping to monitor validation loss with patience of 3 epochs
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=3,          # Stop training after 3 epochs with no improvement
    restore_best_weights=True  # Restore model weights from the epoch with the best validation loss
)

epochs = 50  

# Training with early stopping
for X_train_full, X_val, X_test, y_train_full, y_val, y_test in create_cross_validation_splits(sequences, labels, n_splits=4):
    history = model.fit(
        X_train_full, y_train_full,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=32,
        class_weight=class_weight_dict,  # Handle class imbalance
        callbacks=[early_stopping]  # Include early stopping
    )

test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy}")

Epoch 1/50


2024-10-28 23:53:47.970948: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


12707/12707 ━━━━━━━━━━━━━━━━━━━━ 107s 8ms/step - accuracy: 0.5126 - loss: 0.6871 - val_accuracy: 0.4194 - val_loss: 0.7179
Epoch 2/50
12707/12707 ━━━━━━━━━━━━━━━━━━━━ 108s 8ms/step - accuracy: 0.4807 - loss: 0.6867 - val_accuracy: 0.4943 - val_loss: 0.6756
Epoch 3/50
12707/12707 ━━━━━━━━━━━━━━━━━━━━ 104s 8ms/step - accuracy: 0.4941 - loss: 0.6842 - val_accuracy: 0.6521 - val_loss: 0.6513
Epoch 4/50
12707/12707 ━━━━━━━━━━━━━━━━━━━━ 104s 8ms/step - accuracy: 0.5174 - loss: 0.6797 - val_accuracy: 0.6534 - val_loss: 0.6300
Epoch 5/50
12707/12707 ━━━━━━━━━━━━━━━━━━━━ 103s 8ms/step - accuracy: 0.5383 - loss: 0.6752 - val_accuracy: 0.4881 - val_loss: 0.6883
Epoch 6/50
12707/12707 ━━━━━━━━━━━━━━━━━━━━ 103s 8ms/step - accuracy: 0.5329 - loss: 0.6739 - val_accuracy: 0.5596 - val_loss: 0.6614
Epoch 7/50
12707/12707 ━━━━━━━━━━━━━━━━━━━━ 106s 8ms/step - accuracy: 0.5383 - loss: 0.6706 - val_accuracy: 0.4730 - val_loss: 0.7015
Epoch 1/50
12707/12707 ━━━━━━━━━━━━━━━━━━━━ 104s 8ms/step - accuracy: 0.5

## Class imblance - solve by using Smoke oversampling 

In [36]:

# Split the sequences and labels into training, validation, and testing sets
X_train_full, X_test, y_train_full, y_test = train_test_split(sequences, labels, test_size=0.2, random_state=42)

# Reshape data into 2D for SMOTE (reshape to [n_samples, window_size * n_features])
n_samples, window_size, n_features = X_train_full.shape
X_train_reshaped = X_train_full.reshape((n_samples, window_size * n_features))

# Appling SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_reshaped, y_train_full)

# Reshape data back into 3D for training (reshape to [n_samples, window_size, n_features])
X_train_resampled = X_train_resampled.reshape((X_train_resampled.shape[0], window_size, n_features))

# Split the resampled training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_resampled, y_train_resampled, test_size=0.25, random_state=42)


In [37]:
X_train

array([[[0.93150685, 0.00895778, 0.09849699, ..., 0.        ,
         0.        , 0.        ],
        [0.93150685, 0.00860551, 0.09849699, ..., 0.        ,
         0.        , 0.        ],
        [0.93150685, 0.00810226, 0.09849699, ..., 0.        ,
         0.        , 0.        ],
        [0.93150685, 0.00860551, 0.09849699, ..., 0.        ,
         0.        , 0.        ],
        [0.93150685, 0.00830356, 0.09819639, ..., 0.        ,
         0.        , 0.        ]],

       [[0.64383562, 0.0088068 , 0.09849699, ..., 0.0232288 ,
         0.03894298, 0.6       ],
        [0.64383562, 0.00865583, 0.09849699, ..., 0.0232288 ,
         0.03894298, 0.6       ],
        [0.64383562, 0.0088068 , 0.09849699, ..., 0.0232288 ,
         0.03894298, 0.6       ],
        [0.64383562, 0.00870615, 0.09829659, ..., 0.0232288 ,
         0.03894298, 0.6       ],
        [0.64383562, 0.00870615, 0.09829659, ..., 0.0232288 ,
         0.03894298, 0.6       ]],

       [[0.57534247, 0.00910875, 0.0

In [38]:
# Normalize the data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_val = scaler.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)


## LSTM Model with 2 hidden layer 

In [39]:
# Define the LSTM model
model1 = Sequential()

# Add an LSTM layer
model1.add(LSTM(units=64, return_sequences=True, input_shape=(window_size, sequences.shape[2])))
model1.add(LSTM(units=64, return_sequences=False))

# Add a Dense output layer for binary classification
model1.add(Dense(1, activation='sigmoid'))

# Compile the model
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the model summary
model1.summary()

/Users/sadhikrupa/Library/Python/3.9/lib/python/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 5, 64)          │        19,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 52,289 (204.25 KB)

 Trainable params: 52,289 (204.25 KB)

 Non-trainable params: 0 (0.00 B)

In [40]:

# Define early stopping to monitor validation loss with patience of 3 epochs
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model with early stopping
history1 = model1.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=32,
    callbacks=[early_stopping]
)

# After training, evaluate the model on the test set
test_loss, test_accuracy = model1.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy}")

Epoch 1/50
21191/21191 ━━━━━━━━━━━━━━━━━━━━ 244s 11ms/step - accuracy: 0.5479 - loss: 0.6863 - val_accuracy: 0.5552 - val_loss: 0.6827
Epoch 2/50
21191/21191 ━━━━━━━━━━━━━━━━━━━━ 244s 11ms/step - accuracy: 0.5618 - loss: 0.6801 - val_accuracy: 0.5611 - val_loss: 0.6765
Epoch 3/50
21191/21191 ━━━━━━━━━━━━━━━━━━━━ 245s 12ms/step - accuracy: 0.5764 - loss: 0.6713 - val_accuracy: 0.5871 - val_loss: 0.6638
Epoch 4/50
21191/21191 ━━━━━━━━━━━━━━━━━━━━ 244s 12ms/step - accuracy: 0.5920 - loss: 0.6592 - val_accuracy: 0.6067 - val_loss: 0.6452
Epoch 5/50
21191/21191 ━━━━━━━━━━━━━━━━━━━━ 244s 11ms/step - accuracy: 0.6139 - loss: 0.6406 - val_accuracy: 0.6409 - val_loss: 0.6201
Epoch 6/50
21191/21191 ━━━━━━━━━━━━━━━━━━━━ 244s 12ms/step - accuracy: 0.6439 - loss: 0.6152 - val_accuracy: 0.6746 - val_loss: 0.5834
Epoch 7/50
21191/21191 ━━━━━━━━━━━━━━━━━━━━ 244s 12ms/step - accuracy: 0.6750 - loss: 0.5845 - val_accuracy: 0.6969 - val_loss: 0.5639
Epoch 8/50
21191/21191 ━━━━━━━━━━━━━━━━━━━━ 244s 12ms/s

In [58]:
model1.save("Lstm.h5")

In [41]:
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve, auc, confusion_matrix, classification_report

In [48]:

# Step 1: Calculate Probabilities and Predict Sepsis
y_pred_prob = model1.predict(X_test)
threshold = 0.5  # Adjust threshold if necessary
y_pred = (y_pred_prob >= threshold).astype(int)


4518/4518 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step


## Confusion matrix

In [49]:
# Step 2: Plot the Confusion Matrix with Plotly
conf_matrix = confusion_matrix(y_test, y_pred)
fig_conf_matrix = px.imshow(
    conf_matrix,
    labels=dict(x="Predicted Label", y="True Label", color="Count"),
    x=['No Sepsis', 'Sepsis'],
    y=['No Sepsis', 'Sepsis'],
    title="Confusion Matrix",
    color_continuous_scale='Blues',
    text_auto=True 
)
fig_conf_matrix.show()

In [50]:
# ROC AUC Score
roc_auc = roc_auc_score(y_test, y_pred_prob)
print(f"ROC AUC Score: {roc_auc}")

ROC AUC Score: 0.9037487573497903


## Roc Auc curve 

In [53]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)
fig_roc = go.Figure()
fig_roc.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name=f'ROC Curve (AUC = {roc_auc:.2f})', line=dict(color='darkorange', width=2)))
fig_roc.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Random', line=dict(color='navy', width=2, dash='dash')))

fig_roc.update_layout(
    title="Receiver Operating Characteristic (ROC) Curve",
    xaxis_title="False Positive Rate",
    yaxis_title="True Positive Rate",
    template="plotly_white",
    width=1000,  # Width in pixels
    height=800   # Height in pixels
)

# Display the figure
fig_roc.show()

## Model training and Loss Accuracy 

In [57]:

# Step 4: Plot Training and Validation Loss
fig_loss = go.Figure()
fig_loss.add_trace(go.Scatter(y=history1.history['loss'], mode='lines+markers', name='Training Loss', line=dict(color='blue')))
fig_loss.add_trace(go.Scatter(y=history1.history['val_loss'], mode='lines+markers', name='Validation Loss', line=dict(color='red')))
fig_loss.update_layout(
    title="Training and Validation Loss Over Epochs",
    xaxis_title="Epoch",
    yaxis_title="Loss",
    template="plotly_white",
    width=1000,  # Width in pixels
    height=800   # Height in pixels
)
fig_loss.show()

# Step 5: Plot Model Accuracy
fig_accuracy = go.Figure()
fig_accuracy.add_trace(go.Scatter(y=history1.history['accuracy'], mode='lines+markers', name='Training Accuracy', line=dict(color='green')))
fig_accuracy.add_trace(go.Scatter(y=history1.history['val_accuracy'], mode='lines+markers', name='Validation Accuracy', line=dict(color='orange')))
fig_accuracy.update_layout(
    title="Training and Validation Accuracy Over Epochs",
    xaxis_title="Epoch",
    yaxis_title="Accuracy",
    template="plotly_white",
    width=1000,  # Width in pixels
    height=800   # Height in pixels
)
fig_accuracy.show()


## Classification Report

In [64]:
print("Classification Report:")
print(classification_report(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.81      0.87    112961
           1       0.56      0.84      0.67     31613

    accuracy                           0.82    144574
   macro avg       0.75      0.83      0.77    144574
weighted avg       0.86      0.82      0.83    144574

